In [19]:
import sqlalchemy as db
import pandas as pd
import collections
import math
import operator
import collections
import math
import operator


def getCurrentAmbulanceDistribution(trips, facilities):
    '''Retrieves the current distribution of ambulances accross the country'''
    og_name = {}
    mapping = {}
    for hospital, group in trips.groupby(['Dispatcher_HospitalName']):
        abbreviation = hospital.split()[0].lower()
        
        if abbreviation == 'JDWNRH':
            abbreviation = 'JDWNR'
        
        og_name[abbreviation] = hospital
        mapping[abbreviation] = len(group['Dispatcher_AmbulanceNo'].unique())
        
    def getAmbulanceCount(row, mapping):
        
        if row['hospital'] in mapping.keys():
            return mapping[row['hospital']]
        else:
            return 0


    facilities['ambulance_count'] = facilities.apply(getAmbulanceCount, mapping=mapping, axis=1)
#     print(facilities)
     # TODO get facilities that are not mapped in the excel sheet and have resources
    missing_hospital_address = []
    
    for facility in mapping.keys():
                
        # find hospital entry on the spreadsheets
        hospital = facilities.loc[facilities['hospital'] == facility]

        if len(hospital) == 0:
            missing_hospital_address.append(og_name[facility])

    print(missing_hospital_address)
    
    print("Ambulances mapped %i"%(sum(facilities['ambulance_count'])))
    return facilities


def loadData():
    # load trips
    engine = db.create_engine('postgresql://localhost/bhutan', echo=False)
    connection = engine.connect()
    metadata = db.MetaData()
    trips = db.Table('er_trips', metadata, autoload=True, autoload_with=engine)
    query = db.select([trips])\
                    .where(trips.c.OutgoingTrip_DistanceTravelled > 0.3)\
                    .where(trips.c.OutgoingTrip_TripDuration < 120 * 60)\
                   
    ResultProxy = connection.execute(query)
    ResultSet = ResultProxy.fetchall()
    df = pd.DataFrame(ResultSet)
    df.columns = ResultSet[0].keys()
    
    print(min(df['OutgoingTrip_TripDuration']))
    print(len(df))
    # load facilities
    facilities = db.Table('facilities', metadata, autoload=True, autoload_with=engine)

    query = db.select([facilities])

    ResultProxy = connection.execute(query)
    ResultSet = ResultProxy.fetchall()
    df_facilities = pd.DataFrame(ResultSet)
    df_facilities.columns = ResultSet[0].keys()
    df_facilities = df_facilities.rename(columns={"hospital / bhu": "hospital"})

    df_facilities['raw'] = df_facilities.hospital
    df_facilities.hospital = df_facilities.hospital.str.split().str[0].str.lower()
    df_facilities = getCurrentAmbulanceDistribution(df, df_facilities)
    
    ambulances = len(df['Dispatcher_AmbulanceNo'].unique())
    hospitals = len(df['Dispatcher_HospitalName'].unique())

    print("Total ambulances: %i"%(ambulances))
    print("Total hospitals: %i"%(hospitals))
    #distribution = df['Dispatcher_AmbulanceNo'].value_counts()
#     print("Current distribution: ")
#     print(distribution)

    # load facilities
    return df, df_facilities


    
events, facilities = loadData()

        

149.0
2972
['Dewathang Military Hospital ', 'Eusa BHU ', 'Haa Hospital', 'Jomotshangkha BHU ', 'Khateykha  BHU II', 'Mongar ERRH', 'Rangjung  BHU', 'Wangdi Chholing Hospital ']
Ambulances mapped 90
Total ambulances: 103
Total hospitals: 63


In [13]:

def getBaseHospitalName(name):
    return name.split()[0].lower()

def historicalAnalysis(events, facilities):
    Candidate = collections.namedtuple('Candidate', 'hospital distance')

    def haversine(coord1, coord2):
        R = 6371  # Earth radius in km
        lat1, lon1 = coord1
        lat2, lon2 = coord2
        
        phi1, phi2 = math.radians(lat1), math.radians(lat2) 
        dphi = math.radians(lat2 - lat1)
        dlambda = math.radians(lon2 - lon1)
        
        a = math.sin(dphi/2)**2 + \
            math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
        
        return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

    def calculateHospitalDistance(hospital, event):
        hospital_coords = hospital.latitude, hospital.longitude
        event_coords = event.OutgoingTrip_LatEnd, event.OutgoingTrip_LngEnd     
        return Candidate(hospital=hospital.raw, distance=haversine(hospital_coords, event_coords))
    
    assignments = pd.DataFrame(data={'Assigned_Hospital': [], 
                                     'Ideal_Hospital': [], 
                                     'Distance_Hospital': [], 
                                     'Distance_Ideal': [], 
                                     'Delta': []})
    total = len(events)
    
    for i, event in events.iterrows():
        print("Simulating event %i/%i"%(i + 1, total))
        
        # get hospital that was being called
        distances = facilities.apply(calculateHospitalDistance, event=event, axis=1)

        # get hospital with ambulances closest 
        closest_facility = min(distances, key=operator.itemgetter(1))
        
        # get distance to dispatcher hospital name
        assigned_facility = Candidate(hospital=event.Dispatcher_HospitalName,
                                      distance=haversine([event.OutgoingTrip_LatStart, event.OutgoingTrip_LngStart], 
                                      [event.OutgoingTrip_LatEnd, event.OutgoingTrip_LngEnd]))
        
        delta = 0
        # ignore possible marginal differences
        if assigned_facility.distance - closest_facility.distance >= 1 and assigned_facility.hospital != closest_facility.hospital:
            delta = assigned_facility.distance - closest_facility.distance
        
        assignments = pd.concat([assignments, 
                                 pd.DataFrame(data={'Assigned_Hospital': [assigned_facility.hospital], 
                                         'Ideal_Hospital': [closest_facility.hospital], 
                                         'Distance_Hospital': [assigned_facility.distance], 
                                         'Distance_Ideal': [closest_facility.hospital], 
                                         'Delta': [delta]})])
   
    return assignments


assignments = historicalAnalysis(events, facilities)



Simulating event 1/2972
Simulating event 2/2972
Simulating event 3/2972
Simulating event 4/2972
Simulating event 5/2972
Simulating event 6/2972
Simulating event 7/2972
Simulating event 8/2972
Simulating event 9/2972
Simulating event 10/2972
Simulating event 11/2972
Simulating event 12/2972
Simulating event 13/2972
Simulating event 14/2972
Simulating event 15/2972
Simulating event 16/2972
Simulating event 17/2972
Simulating event 18/2972
Simulating event 19/2972
Simulating event 20/2972
Simulating event 21/2972
Simulating event 22/2972
Simulating event 23/2972
Simulating event 24/2972
Simulating event 25/2972
Simulating event 26/2972
Simulating event 27/2972
Simulating event 28/2972
Simulating event 29/2972
Simulating event 30/2972
Simulating event 31/2972
Simulating event 32/2972
Simulating event 33/2972
Simulating event 34/2972
Simulating event 35/2972
Simulating event 36/2972
Simulating event 37/2972
Simulating event 38/2972
Simulating event 39/2972
Simulating event 40/2972
Simulatin

Simulating event 322/2972
Simulating event 323/2972
Simulating event 324/2972
Simulating event 325/2972
Simulating event 326/2972
Simulating event 327/2972
Simulating event 328/2972
Simulating event 329/2972
Simulating event 330/2972
Simulating event 331/2972
Simulating event 332/2972
Simulating event 333/2972
Simulating event 334/2972
Simulating event 335/2972
Simulating event 336/2972
Simulating event 337/2972
Simulating event 338/2972
Simulating event 339/2972
Simulating event 340/2972
Simulating event 341/2972
Simulating event 342/2972
Simulating event 343/2972
Simulating event 344/2972
Simulating event 345/2972
Simulating event 346/2972
Simulating event 347/2972
Simulating event 348/2972
Simulating event 349/2972
Simulating event 350/2972
Simulating event 351/2972
Simulating event 352/2972
Simulating event 353/2972
Simulating event 354/2972
Simulating event 355/2972
Simulating event 356/2972
Simulating event 357/2972
Simulating event 358/2972
Simulating event 359/2972
Simulating e

Simulating event 643/2972
Simulating event 644/2972
Simulating event 645/2972
Simulating event 646/2972
Simulating event 647/2972
Simulating event 648/2972
Simulating event 649/2972
Simulating event 650/2972
Simulating event 651/2972
Simulating event 652/2972
Simulating event 653/2972
Simulating event 654/2972
Simulating event 655/2972
Simulating event 656/2972
Simulating event 657/2972
Simulating event 658/2972
Simulating event 659/2972
Simulating event 660/2972
Simulating event 661/2972
Simulating event 662/2972
Simulating event 663/2972
Simulating event 664/2972
Simulating event 665/2972
Simulating event 666/2972
Simulating event 667/2972
Simulating event 668/2972
Simulating event 669/2972
Simulating event 670/2972
Simulating event 671/2972
Simulating event 672/2972
Simulating event 673/2972
Simulating event 674/2972
Simulating event 675/2972
Simulating event 676/2972
Simulating event 677/2972
Simulating event 678/2972
Simulating event 679/2972
Simulating event 680/2972
Simulating e

Simulating event 964/2972
Simulating event 965/2972
Simulating event 966/2972
Simulating event 967/2972
Simulating event 968/2972
Simulating event 969/2972
Simulating event 970/2972
Simulating event 971/2972
Simulating event 972/2972
Simulating event 973/2972
Simulating event 974/2972
Simulating event 975/2972
Simulating event 976/2972
Simulating event 977/2972
Simulating event 978/2972
Simulating event 979/2972
Simulating event 980/2972
Simulating event 981/2972
Simulating event 982/2972
Simulating event 983/2972
Simulating event 984/2972
Simulating event 985/2972
Simulating event 986/2972
Simulating event 987/2972
Simulating event 988/2972
Simulating event 989/2972
Simulating event 990/2972
Simulating event 991/2972
Simulating event 992/2972
Simulating event 993/2972
Simulating event 994/2972
Simulating event 995/2972
Simulating event 996/2972
Simulating event 997/2972
Simulating event 998/2972
Simulating event 999/2972
Simulating event 1000/2972
Simulating event 1001/2972
Simulating

Simulating event 1275/2972
Simulating event 1276/2972
Simulating event 1277/2972
Simulating event 1278/2972
Simulating event 1279/2972
Simulating event 1280/2972
Simulating event 1281/2972
Simulating event 1282/2972
Simulating event 1283/2972
Simulating event 1284/2972
Simulating event 1285/2972
Simulating event 1286/2972
Simulating event 1287/2972
Simulating event 1288/2972
Simulating event 1289/2972
Simulating event 1290/2972
Simulating event 1291/2972
Simulating event 1292/2972
Simulating event 1293/2972
Simulating event 1294/2972
Simulating event 1295/2972
Simulating event 1296/2972
Simulating event 1297/2972
Simulating event 1298/2972
Simulating event 1299/2972
Simulating event 1300/2972
Simulating event 1301/2972
Simulating event 1302/2972
Simulating event 1303/2972
Simulating event 1304/2972
Simulating event 1305/2972
Simulating event 1306/2972
Simulating event 1307/2972
Simulating event 1308/2972
Simulating event 1309/2972
Simulating event 1310/2972
Simulating event 1311/2972
S

Simulating event 1586/2972
Simulating event 1587/2972
Simulating event 1588/2972
Simulating event 1589/2972
Simulating event 1590/2972
Simulating event 1591/2972
Simulating event 1592/2972
Simulating event 1593/2972
Simulating event 1594/2972
Simulating event 1595/2972
Simulating event 1596/2972
Simulating event 1597/2972
Simulating event 1598/2972
Simulating event 1599/2972
Simulating event 1600/2972
Simulating event 1601/2972
Simulating event 1602/2972
Simulating event 1603/2972
Simulating event 1604/2972
Simulating event 1605/2972
Simulating event 1606/2972
Simulating event 1607/2972
Simulating event 1608/2972
Simulating event 1609/2972
Simulating event 1610/2972
Simulating event 1611/2972
Simulating event 1612/2972
Simulating event 1613/2972
Simulating event 1614/2972
Simulating event 1615/2972
Simulating event 1616/2972
Simulating event 1617/2972
Simulating event 1618/2972
Simulating event 1619/2972
Simulating event 1620/2972
Simulating event 1621/2972
Simulating event 1622/2972
S

Simulating event 1897/2972
Simulating event 1898/2972
Simulating event 1899/2972
Simulating event 1900/2972
Simulating event 1901/2972
Simulating event 1902/2972
Simulating event 1903/2972
Simulating event 1904/2972
Simulating event 1905/2972
Simulating event 1906/2972
Simulating event 1907/2972
Simulating event 1908/2972
Simulating event 1909/2972
Simulating event 1910/2972
Simulating event 1911/2972
Simulating event 1912/2972
Simulating event 1913/2972
Simulating event 1914/2972
Simulating event 1915/2972
Simulating event 1916/2972
Simulating event 1917/2972
Simulating event 1918/2972
Simulating event 1919/2972
Simulating event 1920/2972
Simulating event 1921/2972
Simulating event 1922/2972
Simulating event 1923/2972
Simulating event 1924/2972
Simulating event 1925/2972
Simulating event 1926/2972
Simulating event 1927/2972
Simulating event 1928/2972
Simulating event 1929/2972
Simulating event 1930/2972
Simulating event 1931/2972
Simulating event 1932/2972
Simulating event 1933/2972
S

Simulating event 2201/2972
Simulating event 2202/2972
Simulating event 2203/2972
Simulating event 2204/2972
Simulating event 2205/2972
Simulating event 2206/2972
Simulating event 2207/2972
Simulating event 2208/2972
Simulating event 2209/2972
Simulating event 2210/2972
Simulating event 2211/2972
Simulating event 2212/2972
Simulating event 2213/2972
Simulating event 2214/2972
Simulating event 2215/2972
Simulating event 2216/2972
Simulating event 2217/2972
Simulating event 2218/2972
Simulating event 2219/2972
Simulating event 2220/2972
Simulating event 2221/2972
Simulating event 2222/2972
Simulating event 2223/2972
Simulating event 2224/2972
Simulating event 2225/2972
Simulating event 2226/2972
Simulating event 2227/2972
Simulating event 2228/2972
Simulating event 2229/2972
Simulating event 2230/2972
Simulating event 2231/2972
Simulating event 2232/2972
Simulating event 2233/2972
Simulating event 2234/2972
Simulating event 2235/2972
Simulating event 2236/2972
Simulating event 2237/2972
S

Simulating event 2515/2972
Simulating event 2516/2972
Simulating event 2517/2972
Simulating event 2518/2972
Simulating event 2519/2972
Simulating event 2520/2972
Simulating event 2521/2972
Simulating event 2522/2972
Simulating event 2523/2972
Simulating event 2524/2972
Simulating event 2525/2972
Simulating event 2526/2972
Simulating event 2527/2972
Simulating event 2528/2972
Simulating event 2529/2972
Simulating event 2530/2972
Simulating event 2531/2972
Simulating event 2532/2972
Simulating event 2533/2972
Simulating event 2534/2972
Simulating event 2535/2972
Simulating event 2536/2972
Simulating event 2537/2972
Simulating event 2538/2972
Simulating event 2539/2972
Simulating event 2540/2972
Simulating event 2541/2972
Simulating event 2542/2972
Simulating event 2543/2972
Simulating event 2544/2972
Simulating event 2545/2972
Simulating event 2546/2972
Simulating event 2547/2972
Simulating event 2548/2972
Simulating event 2549/2972
Simulating event 2550/2972
Simulating event 2551/2972
S

Simulating event 2829/2972
Simulating event 2830/2972
Simulating event 2831/2972
Simulating event 2832/2972
Simulating event 2833/2972
Simulating event 2834/2972
Simulating event 2835/2972
Simulating event 2836/2972
Simulating event 2837/2972
Simulating event 2838/2972
Simulating event 2839/2972
Simulating event 2840/2972
Simulating event 2841/2972
Simulating event 2842/2972
Simulating event 2843/2972
Simulating event 2844/2972
Simulating event 2845/2972
Simulating event 2846/2972
Simulating event 2847/2972
Simulating event 2848/2972
Simulating event 2849/2972
Simulating event 2850/2972
Simulating event 2851/2972
Simulating event 2852/2972
Simulating event 2853/2972
Simulating event 2854/2972
Simulating event 2855/2972
Simulating event 2856/2972
Simulating event 2857/2972
Simulating event 2858/2972
Simulating event 2859/2972
Simulating event 2860/2972
Simulating event 2861/2972
Simulating event 2862/2972
Simulating event 2863/2972
Simulating event 2864/2972
Simulating event 2865/2972
S

In [14]:
def calculateMetrics(assignments):
    print(sum(assignments['Delta']))
    print(len(assignments['Delta']))
    optimal = sum([1 if delta == 0 else 0 for delta in assignments['Delta']]) 
    avg_delta = sum(assignments['Delta']) / float(optimal)
    
    total = len(assignments)
    print("Average kilometers added by suboptimal BHU assignment %f"%(avg_delta))
    print("Percentage of optimal assignments %f (%i, %i)"%(float(optimal / total), optimal, total))

calculateMetrics(assignments)

5562.698089078027
2972
Average kilometers added by suboptimal BHU assignment 3.088672
Percentage of optimal assignments 0.605989 (1801, 2972)


In [15]:
assignments.to_csv("test.csv")

In [29]:
import itertools
print(len(facilities))

boxes = len(events['Dispatcher_AmbulanceNo'].unique())
balls = len(facilities)
rng = list(range(balls + 1)) * boxes
print(boxes)
print(balls)
#distributions = set(i for i in itertools.permutations(rng, boxes) if sum(i) == balls)


247
103
247


In [ ]:
# for every season
    # for every distribution
        # for all the cases (assume no overallocation problem)
            # calculate metrics for each (avg response time, % under X minutes)